In [1]:
# https://www.kaggle.com/airbnb/seattle
# 上のサイトからダウンロードしたデータセット

In [1]:
import datetime
import graphviz
import itertools
import os
import platform
import socket
import sys
import time
import warnings
import category_encoders as ce
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_profiling as pdp
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from tqdm import tqdm
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 300)
sys.path.insert(0, '../utils/')
from utils import *

In [32]:
start("airbnb_seattle_listing")

DATE: 2019-04-04 15:53:06.973856
FILE: airbnb_seattle_listing
PID: 1556
HOST: Y-project-11
ENV: Windows-10-10.0.17134-SP0


In [17]:
file_name = "../data/input/airbnb_seattle/listings.csv"
dtypes = {
    "id":"category",
    "listing_url":"category",
    "scrape_id":"int64",
    "last_scraped":"object",
    "name":"object",
    "summary":"object",
    "space":"object",
    "description":"object",
    "experiences_offered":"object",
    "neighborhood_overview":"object",
    "notes":"object",
    "transit":"object",
    "thumbnail_url":"object",
    "medium_url":"object",
    "picture_url":"object",
    "xl_picture_url":"object",
    "host_id":"int64",
    "host_url":"object",
    "host_name":"object",
    "host_since":"object",
    "host_location":"object",
    "host_about":"object",
    "host_response_time":"object",
    "host_response_rate":"object",
    "host_acceptance_rate":"object",
    "host_is_superhost":"object",
    "host_thumbnail_url":"object",
    "host_picture_url":"object",
    "host_neighbourhood":"object",
    "host_listings_count":"float64",
    "host_total_listings_count":"float64",
    "host_verifications":"object",
    "host_has_profile_pic":"object",
    "host_identity_verified":"object",
    "street":"object",
    "neighbourhood":"object",
    "neighbourhood_cleansed":"object",
    "neighbourhood_group_cleansed":"object",
    "city":"object",
    "state":"object",
    "zipcode":"object",
    "market":"object",
    "smart_location":"object",
    "country_code":"object",
    "country":"object",
    "latitude":"float64",
    "longitude":"float64",
    "is_location_exact":"object",
    "property_type":"object",
    "room_type":"object",
    "accommodates":"int64",
    "bathrooms":"float64",
    "bedrooms":"float64",
    "beds":"float64",
    "bed_type":"object",
    "amenities":"object",
    "square_feet":"float64",
    "price":"object",
    "weekly_price":"object",
    "monthly_price":"object",
    "security_deposit":"object",
    "cleaning_fee":"object",
    "guests_included":"int64",
    "extra_people":"object",
    "minimum_nights":"int64",
    "maximum_nights":"int64",
    "calendar_updated":"object",
    "has_availability":"object",
    "availability_30":"int64",
    "availability_60":"int64",
    "availability_90":"int64",
    "availability_365":"int64",
    "calendar_last_scraped":"object",
    "number_of_reviews":"int64",
    "first_review":"object",
    "last_review":"object",
    "review_scores_rating":"float64",
    "review_scores_accuracy":"float64",
    "review_scores_cleanliness":"float64",
    "review_scores_checkin":"float64",
    "review_scores_communication":"float64",
    "review_scores_location":"float64",
    "review_scores_value":"float64",
    "requires_license":"object",
    "license":"float64",
    "jurisdiction_names":"object",
    "instant_bookable":"object",
    "cancellation_policy":"object",
    "require_guest_profile_picture":"object",
    "require_guest_phone_verification":"object",
    "calculated_host_listings_count":"int64",
    "reviews_per_month":"float64"
}
url_col = {
    "listing_url",
    "thumbnail_url",
    "medium_url",
    "picture_url",
    "xl_picture_url",
    "host_url",
    "host_thumbnail_url",
    "host_picture_url"
}
datetime_col = {
    "last_scraped",
    "host_since",
    "calendar_last_scraped",
    "first_review",
    "last_review"
}
price_col = {
    "price",
    "weekly_price",
    "monthly_price",
    "security_deposit",
    "extra_people"
}
with timer("read"):
    df = pd.read_csv(file_name, header=0, index_col=None, sep=',', dtype=dtypes)

[read] start
[read] done in 0 s


In [19]:
# 前処理
# datetime 型に変更
for c in datetime_col:
    df[c] = pd.to_datetime(df[c])
# 文字列処理
for c in price_col:
    musk = ~df[c].isnull()
    df[c][musk] = df[c][musk].str.replace("$","")
    df[c][musk] = df[c][musk].str.replace(",","")
    df[c] = df[c].astype("float32")

In [20]:
df.shape

(3818, 92)

In [21]:
df.head(3)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,NaN,NaN,NaN,NaN,https://a1.muscache.com/ac/pictures/67560560/c...,NaN,956883,https://www.airbnb.com/users/show/956883,Maija,2011-08-11,"Seattle, Washington, United States","I am an artist, interior designer, and run a s...",within a few hours,96%,100%,f,https://a0.muscache.com/ac/users/956883/profil...,https://a0.muscache.com/ac/users/956883/profil...,Queen Anne,3.0,3.0,"['email', 'phone', 'reviews', 'kba']",t,t,"Gilman Dr W, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,98119,Seattle,"Seattle, WA",US,United States,47.636289,-122.371025,t,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",NaN,85.0,NaN,NaN,NaN,NaN,2,5.0,1,365,4 weeks ago,t,14,41,71,346,2016-01-04,207,2011-11-01,2016-01-02,95.0,10.0,10.0,10.0,10.0,9.0,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",What's up with the free pillows? Our home was...,"Convenient bus stops are just down the block, ...",https://a0.muscache.com/ac/pictures/14409893/f...,https://a0.muscache.com/im/pictures/14409893/f...,https://a0.muscache.com/ac/pictures/14409893/f...,https://a0.muscache.com/ac/pictures/14409893/f...,5177328,https://www.airbnb.com/users/show/5177328,Andrea,2013-02-21,"Seattle, Washington, United States",Living east coast/left coast/overseas. Time i...,within an hour,98%,100%,t,https://a0.muscache.com/ac/users/5177328/profi...,https://a0.muscache.com/ac/users/5177328/profi...,Queen Anne,6.0,6.0,"['email', 'phone', 'facebook', 'linkedin', 're...",t,t,"7th Avenue West, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,98119,Seattle,"Seattle, WA",US,United States,47.639123,-122.365666,t,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",NaN,150.0,1000.0,3000.0,100.0,$40.00,1,0.0,2,90,today,t,13,13,16,291,2016-01-04,43,2013-08-19,2015-12-29,96.0,10.0,10.0,10.0,10.0,10.0,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New mo

In [31]:
df["host_thumbnail_url"].iloc[1]

'https://a0.muscache.com/ac/users/5177328/profile_pic/1361540767/original.jpg?interpolation=lanczos-none&crop=w:w;*,*&crop=h:h;*,*&resize=50:*&output-format=jpg&output-quality=70'

In [24]:
df.dtypes

id                                        category
listing_url                               category
scrape_id                                    int64
last_scraped                        datetime64[ns]
name                                        object
summary                                     object
space                                       object
description                                 object
experiences_offered                         object
neighborhood_overview                       object
notes                                       object
transit                                     object
thumbnail_url                               object
medium_url                                  object
picture_url                                 object
xl_picture_url                              object
host_id                                      int64
host_url                                    object
host_name                                   object
host_since                     

In [23]:
pdp.ProfileReport(df)

Number of variables,92
Number of observations,3818
Total Missing (%),7.3%
Total size in memory,2.9 MiB
Average record size in memory,805.8 B
Numeric,27
Categorical,43
Date,3
Text (Unique),3
Rejected,16
Distinct count,15


- HPの情報
	- URL （宿紹介ページ、サムネイル、大中小画像）
	- 宿名
	- 概要説明
	- 詳細説明
	- 近所の施設
	- 注記
	- 交通案内
- ホストについて
	- ID
	- HPのURL
	- 開始日
	- 場所
	- プロフィール
	- 返答時間
	- 返答率
	- 受理率
	- 模範ホストフラグ
	- 画像
	- プロフィール認証
	- ストリート
- 場所について
	- ストリート
	- 州
	- 地域
	- 国
	- 緯度経度
- 宿について
	- 宿タイプ（家、アパートなど）
	- 部屋タイプ（プライベート型、共有型など）
	- 収容人数
	- 風呂数
	- 寝室数
	- ベッド数
	- ベッドタイプ（ベッド、ふとんなど）
	- アメニティ（TV、キッチンなど）
	- 面積（平方フィート）
	- 価格
	- ウィークリー価格
	- マンスリー価格
	- 保証金
	- クリーニング料金
	- カレンダーの前回更新日
	- availability_x ... ???
- レビューについて
	- レビュー数
	- 最初のレビュー日
	- 最後のレビュー日
	- レビュースコアレーティング（正確さ、清潔度、チェックイン、コミュニケーション、場所、価格）
	- 今すぐ予約フラグ
	- キャンセル時のポリシー
	- ゲストのプロフィール画像要求フラグ
	- ゲストの電話認証の要求フラグ
	- ホストが保有する宿数 ... ?
	- 1か月あたりレビュー数
	